##3.6 상위 데이터만 추천
>기존 데이터에는 0이 포함되어 있어 정확한 결과를 도출해 낼 수 없었다. 이번에는 상위 몇 개의 데이터만 추출하여 추천해 주는 시스템을 구축해 보자.

In [1]:
print(ratings_arr)
print('========================================================================')
print('0 행 출력\n', ratings_arr[0])
print('0 열만 출력\n', ratings_arr[:, 0])
print('4 열만 출력\n', ratings_arr[:, 4])

NameError: ignored

> 데이터 출력시 []안에 숫자를 넣으면 그 행을 출력한다. 열을 출력하고 싶다면 [:, idx] 형식으로 입력하면 원하는 열을 출력할 수 있다. 위의 코드에서 idx 위치에 0을 입력하면 0번째 열이 나타나게 된다.

In [ ]:
print(ratings_arr)
print('========================================================================')
print('각 행별 오름차순 index 출력')
print(ratings_arr[0, :])
print(np.argsort(ratings_arr[0, :]))
print(ratings_arr[1, :])
print(np.argsort(ratings_arr[1, :]))

>argsort()함수는 입력된 데이터를 정렬한 후 index를 반환한다. 0번째 데이터를 보면 4, 0. ,4, 0, 5로 정렬하게 되면 0, 0, 4, 4, 5가 되며 index만 추출하게 되면 1, 3, 0. 2, 4가 되게 된다. 같은 숫자는 먼저 나온 값을 작은 값으로 인식한다.

In [ ]:
print(ratings_arr)
print('========================================================================')
print('각 행별 내림차순 index 출력')
print(np.argsort(ratings_arr[0, :])[:])
print(np.argsort(ratings_arr[0, :])[:4])
print(np.argsort(ratings_arr[0, :])[:-1])
print(np.argsort(ratings_arr[0, :])[:-2])
print(np.argsort(ratings_arr[0, :])[:-4])
print(np.argsort(ratings_arr[0, :])[:-4:-1])

>출력 결과에서 [:col]에서 col의 값이 양수인 경우 그 위치 전까지 추출하며 음수인 경우 마지막 위치부터 값을 추출할 수 있다. 여기서 다시 [:col:asc] asc 위치에 -1을 입력하게 되면 지금 값을 제외한 내림차순의 값이 추출하게 된다.

In [ ]:
print(ratings_arr)
print('========================================================================')
n=3
top_n_items = np.argsort(ratings_arr[0, :])[:-1-n:-1]
print(ratings_arr[0, :])
print(ratings_arr[0, :][top_n_items])

>top_n_items 상위 n개의 item을 추출하기 위한 index 배열이다. 이전 내용에서 내림차순으로 정렬한 데이터의 index를 추출하여 위치정보를 저장하게 된다. n을 3이라고 가정하면 상위3개의 데이터가 출력되게 되어 4,0,4,0,5의 데이터 중 5,4,4라는 3개의 데이터만 추출했다.

In [ ]:
print('========================================================================')
print(ratings_arr.shape)
pred = np.zeros(ratings_arr.shape)
print(pred)

>zeros함수를 이용하면 0으로 채워진 행렬을 만들게 된다. 위와 같이 shape를 이용하여 raitings_arr의 크기 만한 행렬을 만들 수 있다.

In [ ]:
print('========================================================================')
pred[0, 1]=(item_sim_arr[:, 1][top_n_items].dot(ratings_arr[0, :][top_n_items]))
pred[1, 1]=(item_sim_arr[:, 1][top_n_items].dot(ratings_arr[1, :][top_n_items]))
pred[2, 1]=(item_sim_arr[:, 1][top_n_items].dot(ratings_arr[2, :][top_n_items]))
pred[0, 4]=(item_sim_arr[:, 4][top_n_items].dot(ratings_arr[0, :][top_n_items]))
pred[1, 4]=(item_sim_arr[:, 4][top_n_items].dot(ratings_arr[1, :][top_n_items]))
pred[2, 4]=(item_sim_arr[:, 4][top_n_items].dot(ratings_arr[2, :][top_n_items]))
print(pred)

>위와 같이 예측 평점을 구할 문자를 만들 수 있다.

In [ ]:


print('========================================================================')
print(item_sim_arr[1, :])
print(item_sim_arr[1, :][top_n_items])
print(np.abs(item_sim_arr[1, :][top_n_items]))
print(np.sum(np.abs(item_sim_arr[1, :][top_n_items])))

위와 같이 예측 평점을 구할 분모를 만들 수 있다.

In [ ]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
  pred=np.zeros(ratings_arr.shape)
  
  for col in range(ratings_arr.shape[1]):
    top_n_items = np.argsort(item_sim_arr[:,col])[:-n-1:-1]

    for row in range(ratings_arr.shape[0]):
      pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items])
      pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))

  return pred

ratings_pred = predict_rating_topsim(df_ratings_movies.values, movies_similarity, n=3)
print(ratings_pred)
print(get_mse(ratings_pred,df_ratings_movies.values ))

In [ ]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index = df_ratings_movies.index, columns=df_ratings_movies.columns)
ratings_pred_matrix

>지금까지 내용을 정리하여 위와 같이 함수를 만들 수 있다. 예측 점수를 확인해 보면 이전보다 더 실측점수와 비슷하게 나타난 것을 확인할 수 있다. 그리고 MSE(평균 제곱 오류)도 작게 나온것을 볼 수 있다.

##3.7 개인 추천

In [ ]:
user_rating_id = df_ratings_movies.loc['user1', :]
user_rating_id

In [ ]:
user_rating_id[user_rating_id>0].sort_values(ascending=False)

In [ ]:
test = user_rating_id[user_rating_id>0].sort_values(ascending=False)

>그럼 간단하게 user1이 지금까지 봤었던 영화들을 확인해 보자.

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
  user_rating = ratings_matrix.loc[userId, :]
  already_seen = user_rating[user_rating>0].index
  movies_list = ratings_matrix.columns
  unseen_list = movies_list[~movies_list.isin(already_seen)]

  return unseen_list.tolist()

get_unseen_movies(df_ratings_movies, 'user1')

> 간단하게 지정된 사용자가 안본 영화 목록을 추출해 봤다.

In [ ]:
def recomm_by_userid(pred_df, userId, unseen_list, top_n=10):
  recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

unseen_list = get_unseen_movies(df_ratings_movies, 'user1')
recomm_by_userid(ratings_pred_matrix, 'user1', unseen_list, 3)

>추출한 영화 중 상위 3개만 추출한다.

In [ ]:
recomm_movies = recomm_by_userid(ratings_pred_matrix, 'user1', unseen_list, 3)
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

>dataframe으로 보기 좋게 꾸밀 수 있다.

추후 확인
http://www.tourbigdata.kr/award.asp